In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import statsmodels.api as sm

In [17]:
ticker = 'NVDA'
start_date = '2023-01-01'
end_date = '2023-12-31'

data = yf.download(ticker, start=start_date, end=end_date)
data.columns = data.columns.droplevel(['Ticker'])
data = data[['Close']]


data.head()

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['NVDA']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Price,Close
Date,


In [18]:
pip install yfinance --upgrade

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.52
    Uninstalling yfinance-0.2.52:
      Successfully uninstalled yfinance-0.2.52

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
